In [2]:
import json
def get_text_entities(path):
    with open(path, 'r') as fobj:
        d = json.load(fobj)
    return d

In [3]:
from nltk.tokenize import wordpunct_tokenize, word_tokenize
from nltk import pos_tag
import string
import pandas as pd

In [4]:
def get_label(tokens):
    if len(tokens) == 1:
        return ['B-ing']
    else:
        labels = ['B-ing']
        for t in range(1, len(tokens)):
            labels.append('I-ing')
        return labels

def getO_label(tokens):
    labels = []
    for t in tokens:
        labels.append('O')
    return labels

def label_ingredients(txt, labels):
    tokens = []
    t_lables = []
    prev = 0
    for start,end,_ in labels['entities']:
        pre_str = txt[prev:start]
        pre_tokens = word_tokenize(pre_str)
        pre_labels = getO_label(pre_tokens)
        tokens.extend(pre_tokens)
        t_lables.extend(pre_labels)
        
        cur_str = txt[start:end]
        cur_tokens = word_tokenize(cur_str)
        cur_labels = get_label(cur_tokens)
        tokens.extend(cur_tokens)
        t_lables.extend(cur_labels)
        
        prev = end
        
    return tokens, t_lables 

In [9]:
data = get_text_entities('combined_valid.json')

dfs = []

for i in range(len(data)):
    txt, labels = data[i]

    tokens, annotates  = label_ingredients(txt, labels)
    pos_tags = list(map(lambda x: x[1], pos_tag(tokens)))
    if len(tokens) == 0:
        # nothing here
        print(i)
        print(txt)
        continue
    
    data_dict = {}
    data_dict['Sentence'] = [f'Sentence: {i + 1}'] + [''] * (len(tokens)-1)
    data_dict['Word'] = tokens
    data_dict['POS'] = pos_tags
    data_dict['Tag'] = annotates

    df = pd.DataFrame.from_dict(data_dict, orient='columns')
    dfs.append(df)

118
Puree all ingredients in blender on high until smooth. Serve immediately.
247
Place all ingredients in a blender and blend until smooth.
696
Mix ingredients in microwaveable bowl. Microwave on HIGH 5 minutes or until VELVEETA is completely melted, stirring after 3 minutes.
1040
Puree all ingredients in a blender until smooth.
1308
Puree all ingredients in a blender until smooth.
2862
Blend all ingredients together, very slowly increasing speed from low to high. Blend for 1 1/2 minutes at high.
2906
Place all ingredients into blender and blend on high until smooth.
3174
Combine all ingredients in a large bag and shake to mix. Divide and store in individual baggies for a fast treat.
3651
Add all ingredients to blender. Blend on high until smooth. Enjoy!
4549
Mix ingredients in microwaveable bowl. Microwave on HIGH 5 minutes or until VELVEETA is completely melted, stirring after 3 minutes.


In [10]:
df_merged = pd.concat(dfs)

In [11]:
df_merged.to_csv('combined_valid.csv', index=False)

In [12]:
df_merged.shape

(458258, 4)